In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re, threading
import os
import requests

In [2]:
def scarpe(path):
    options = ChromeOptions()
    options.headless=True
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()
    
    # cookie --> done
    try: 
        cookie = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@class = "cc-compliance"]')))
        if cookie:
            cookie.click()
        else:
            print("no cookies")
    except Exception as e:
        print("no cookies")
    
    
    # clck login
    try:
        # log_card =WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="site-nav__account"]'))) 
        login = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/a[text()="Login"]')))
        if login:
            login.click()
    except Exception as e:
        print("no login")

    # get the username tab
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"
        user_name = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID,'username')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")
    
    # get password tab
    try:
        provided_pass = "Muhssan7865@"
        password = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'password')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 
    
    # get the check done
    try:
        check = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/button[text() = "Sign in"]')))
        if check:
            check.click()
    except Exception as e:
        print(f"No check tab found and the error is {e}")
        
    time.sleep(1)
  
    # number of cars
    try:
        # time.sleep(1)
        cars = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[@class="info-results"]'))).text.strip()
        if cars:
            total_cars = int(cars.split(" ")[-2])
            print(f"{total_cars} number of cars found")
        else:
            print("No cars found")
    except Exception as e:
        print(f"No cars found and error is {e}")
    
    
    # result
    results = []
    car_count = 0
    while car_count < total_cars: #total_cars
        
        try:
            page_cars = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/a[@class="btn btn-secondary btn-icon btn-icon-arrow btn-icon-right"]')))
            
            for i in range(len(page_cars)): # len(page_cars)
                if car_count >= total_cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/a[@class="btn btn-secondary btn-icon btn-icon-arrow btn-icon-right"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    time.sleep(1)
                    page_cars[i].click()

                    details = {}
                    
                    try:
                        title = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="container title-container"]/h1'))).text.strip()
                        if title:
                            details['Title'] = title
                        else:
                            details['Title'] = 'na'
                    except:
                        print("no title")
                    
                    try:
                        lot =WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/h2[@class="h3 detail-lot"]'))).text.strip().split(" ")
                        if lot:
                            details['lot']  = int(lot[1])
                            details['Start Date'] = int(lot[3])
                            details['Start Time'] = int(lot[-1])
                        else:
                            details['Lot'] = 'na'
                            details['Start Date'] = 'na'
                            details['Start Time'] = 'na'
                    except:
                        pass
                        
                    try:
                        det_card =WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/ul[@class="detail-list"]'))) 
                        if det_card:
                            lis = det_card.find_elements(By.TAG_NAME, 'li')
                            for li in lis:
                                det_col = li.find_element(By.XPATH, './/span[@class="item-key"]').text.strip()
                                det_val = li.find_element(By.XPATH, './/span[@class="item-value"]').text.strip()
                                if det_val and det_col:
                                    details[det_col] = det_val
                        else:
                            print("no data")
                    except:
                        pass
                    # images --> done
                    try:
                        # time.sleep(2)
                        img_card =WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="ug-thumbs-strip-inner"])[1]')))
                        if img_card:
                            imgs = WebDriverWait(img_card, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/img[contains(@alt,"Image")]')))
                            imgs_lst = [img.get_attribute("src") for img in imgs]
                            details['Images'] = ", ".join(imgs_lst)
                        else:
                            details['Images'] = 'na'
                    except Exception as e:
                        print(f"No images")
                    
                    try:
                        # time.sleep(2)
                        dmg_img_card =WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="ug-thumbs-strip-inner"])[2]')))
                        if dmg_img_card:
                            dmg_imgs = WebDriverWait(dmg_img_card, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/img[contains(@alt,"Image")]')))
                            dmg_imgs_lst = [dmg_img.get_attribute("src") for dmg_img in dmg_imgs]
                            details['Damaged Images'] = ", ".join(dmg_imgs_lst)
                        else:
                            details['Damaged Images'] = 'na'
                    except Exception as e:
                        print(f"No Damaged_images")
                    
                    try:
                        interior = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '(.//table[@class="table table-striped"])[1]/tbody[1]')))
                        if interior:
                            trs = interior.find_elements(By.TAG_NAME, 'tr')
                            if trs:
                                for tr in trs:
                                    interior_col = tr.find_elements(By.TAG_NAME, 'td')
                                    # interior_val = tr.find_element(By.XPATH, '(.//table[@class="table table-striped"])[1]/tbody[1]/tr/td[2]')
                                    if interior_col:
                                        details[interior_col[0].text.strip()] = interior_col[1].text.strip()                
                    except Exception as e:
                        print("no interior")
                    
                    try:
                        tyre = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '(.//table[@class="table table-striped"])[2]/tbody[1]')))
                        if tyre:
                            tyre_trs = tyre.find_elements(By.TAG_NAME, 'tr')
                            if tyre_trs:
                                for tyre_tr in tyre_trs:
                                    tyre_col = tyre_tr.find_elements(By.TAG_NAME, 'td')
                                    # interior_val = tr.find_element(By.XPATH, '(.//table[@class="table table-striped"])[1]/tbody[1]/tr/td[2]')
                                    if tyre_col:
                                        details[tyre_col[0].text.strip()] = [tyre_col[1].text.strip(), tyre_col[2].text.strip()]                
                    except Exception as e:
                        print("no tyre")
                    
                    try:
                        inspec = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/a[text() ="View PDF"]'))).get_attribute("href")
                        if inspec:
                            details['Inspection Report'] = inspec
                        else:
                            details['Inspection Report'] = 'na'
                    except:
                        pass 
                    
                    try:
                        notes =WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="single-details"])[2]/p'))).text.strip()
                        if notes:
                            details['Notes'] = notes
                        else:
                            details['Notes']  = 'na'
                    except: 
                        pass
                    
                    try:
                        mechs = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/span[@style="font-size: 18px"]')))
                        oks = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/span[@class="float-right"]')))
                        vals = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@style="font-size: 14px"]')))
                        if mechs and oks and vals:
                            # details[mechs]
                            for mech, ok, val in zip(mechs, oks, vals):
                                details[mech.text.strip()] = [ok.text.strip(), val.text.strip()]
                    except Exception as e:
                        print("no mechs")
                        
                    car_count+=1
                    results.append(details)
                    driver.back()
                    
                except Exception as e:
                    print("No car link to click")
                    
                    
            if car_count % 30 == 0: # --> 25%25 == 0 and 50%25== 0 --> issue here
                try:
                    next_link = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '(.//a[@title = "next"])[2]')))
                    if next_link:
                        driver.execute_script("arguments[0].scrollIntoView();", next_link)
                        time.sleep(2)
                        next_link.click()
                        
                        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/a[@class="btn btn-secondary btn-icon btn-icon-arrow btn-icon-right"]')))
                        print("Hit the next button")
                except Exception as e:
                    print("No next found")
                    break
                
        except Exception as e:
            print("No more cars found")
            break
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("shoreham.csv", index=False)
    driver.quit()
path = "https://www.shorehamvehicleauctions.com/auction/137"
scarpe(path)


no cookies
111 number of cars found
no mechs
no mechs
no mechs
no mechs
no mechs
no mechs
No Damaged_images
no interior
no tyre
no mechs
no mechs
no mechs
no mechs
no mechs
no mechs
no mechs
no mechs
no mechs
Hit the next button
no mechs
Hit the next button
Hit the next button
No Damaged_images
no interior
no tyre
no mechs
No Damaged_images
no interior
no tyre
no mechs
No images
No Damaged_images
no interior
no tyre
no mechs
No images
No Damaged_images
no interior
no tyre
no mechs
No images
No Damaged_images
no interior
no tyre
no mechs


In [3]:
df = pd.read_csv("shoreham.csv")
df1 = df.rename(columns={"Registration":"Reg", "Registered":"D.O.R", "Miles":"Mileage", "Fuel":"Fuel Type", "VAT":"VAT Status", "MOT":"MOT Due", "Last Service Miles":"Last service mileage", "Last Service Date":"Last Service"})
df1['Mileage warranted'] = df1['Mileage'].apply(lambda x: x.split(' ')[-1] if isinstance(x, str) and ' ' in x else None)
df1.to_csv("shoreham_data.csv", index=False)

# os.remove("shoreham.csv")

In [4]:
df = pd.read_csv("shoreham_data.csv")
df.columns

Index(['Title', 'lot', 'Make', 'Model', 'Variant', 'Reg', 'D.O.R', 'Mileage',
       'Former Keepers', 'Fuel Type', 'Transmission', 'Colour', 'CC',
       'VAT Status', 'MOT Due', 'V5', 'Keys', 'Service History', 'CAP Clean',
       'CAP Average', 'CAP Below', 'Vendor', 'Images', 'Damaged Images',
       'Carpet condition', 'Seat condition', 'Upholstery condition', 'Odour',
       'Tyre NSF', 'Tyre OSF', 'Tyre NSR', 'Tyre OSR', 'Spare Tyre',
       'Inspection Report', 'Notes', 'Number of Stamps',
       'Last service mileage', 'Last Service', 'Engine runs', 'Brakes',
       'Clutch', 'Gearbox', 'Warning lights', 'Steering', 'Media',
       'Panoramic roof/sunroof/convertible', 'Service indicator', 'Windows',
       'Central locking', 'Mileage warranted'],
      dtype='object')

### images and reports

In [5]:
from PIL import Image, ImageDraw, ImageFont
def add_watermark_to_image(image_path, text="Sourced from Shoreham Vehicle Auctions"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        # Load font
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()

        # Calculate text size and position
        margin = 10
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        # Draw semi-transparent background box
        box_width = text_width + 2 * margin
        box_height = text_height + 2 * margin
        draw.rectangle([x - margin, y - margin, x - margin + box_width, y - margin + box_height],fill=(0, 0, 0, 160))

        # Draw watermark text
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 200))

        # Merge and save
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"Watermark added to {image_path}")

    except Exception as e:
        print(f"Failed to watermark {image_path}: {e}")


In [6]:
from urllib.parse import urlparse, urljoin

df = pd.read_csv("shoreham_data.csv")
reg_img = df[['Reg', "Images", "Damaged Images"]]
reports = df[['Reg', "Inspection Report"]]


def download_images(data, main_folder="Images"):
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # Loop through every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Reg"]  # Separate reg nums
        
        # Split image URLs by commas for both normal and damaged_images
        image_urls = row["Images"].split(", ") if pd.notna(row["Images"]) else []
        damaged_urls = row["Damaged Images"].split(", ") if pd.notna(row["Damaged Images"]) else []
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no)  # Combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True)  # Create the subfolder
        
        # Create a subfolder for images inside the registration folder
        images_folder = os.path.join(reg_folder, "Original_images")  # Folder for regular images
        os.makedirs(images_folder, exist_ok=True)  # Ensure the "Images" folder exists

        # Create a subfolder for damaged_images inside the registration folder
        damaged_folder = os.path.join(reg_folder, "Damaged Images")  # Folder for damaged_images
        os.makedirs(damaged_folder, exist_ok=True)  # Ensure the "Damaged" folder exists

        # Download regular images
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")):  # Check if the URL starts with 'http' or 'https'
                url = urljoin("https://", url)  # Add 'https://' if missing

            # Parse the URL
            parsed_url = urlparse(url)

            # Check if the parsed URL is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}")
                continue
            
            # Try downloading the image
            try:
                # Download the image
                response = requests.get(url, stream=True)
                response.raise_for_status()  # Raises an exception if the HTTP request fails

                # Extract the file name from the URL
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Check for valid file extensions
                file_extension = file_name.split(".")[-1]
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"  # Assign a default name if extension is invalid
                
                # Construct the full path for the file inside the "Images" folder
                full_file_name = os.path.join(images_folder, f"{reg_no}_{idx + 1}.jpg")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

                # Add watermark
                add_watermark_to_image(full_file_name)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

        # Download damaged_images
        for idx, url in enumerate(damaged_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url)  # Add 'https://' if missing

            # Parse the URL
            parsed_url = urlparse(url)

            # Check if the parsed URL is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}")
                continue
            
            # Try downloading the image
            try:
                # Download the image
                response = requests.get(url, stream=True)
                response.raise_for_status()  # Raises an exception if the HTTP request fails

                # Extract the file name from the URL
                file_name = os.path.basename(parsed_url.path) or f"damaged_{idx + 1}.jpg"

                # Check for valid file extensions
                file_extension = file_name.split(".")[-1]
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"damaged_{idx + 1}.jpg"  # Assign a default name if extension is invalid
                
                # Construct the full path for the file inside the "Damaged" folder
                full_file_name = os.path.join(damaged_folder, f"{reg_no}_{idx + 1}.jpg")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

                # Add watermark
                add_watermark_to_image(full_file_name)

                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# download reports 
def download_reports(data, main_folder="Reports"):
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # Loop through every row to get the info
    for i, row in data.iterrows():
        reg_no = row["Reg"]  # Separate reg nums
        Inspection_report_urls = row["Inspection Report"]
        # Assured_report_urls = row["Other Report"]
        
        # Create a subfolder for Inspection Report 
        Condition_report_folder = os.path.join(main_folder, "Inspection Report")  # Folder for regular images
        os.makedirs(Condition_report_folder, exist_ok=True)  # Ensure the "Images" folder exists
        
        # Create a subfolder for Other Report
        # Assured_report_folder = os.path.join(main_folder, "Other Report")  # Folder for damaged_images
        # os.makedirs(Assured_report_folder, exist_ok=True)  # Ensure the "Damaged" folder exists
        
        

        # Check if the Inspection_report_urls is missing
        if not Inspection_report_urls or pd.isna(Inspection_report_urls):
            print(f"Missing Inspection Report of {reg_no}")
            # continue
        else:
        
            if not Inspection_report_urls.startswith(("http://", "https://")):  # Check if the URL starts with 'http' or 'https'
                Inspection_report_urls = urljoin("https://", Inspection_report_urls)  # Add 'https://' if missing

            # Parse the URL
            parsed_url = urlparse(Inspection_report_urls)

            # Check if the parsed URL is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {Inspection_report_urls}")
                continue
            
            # Try downloading the image
            try:
                # Download the image
                response = requests.get(Inspection_report_urls, stream=True)
                response.raise_for_status()  # Raises an exception if the HTTP request fails

                # Extract the file name from the URL
                file_name = os.path.basename(parsed_url.path) or f"Inspection Report.pdf"

                # Check for valid file extensions
                file_extension = file_name.split(".")[-1]
                if file_extension not in ["pdf", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"Inspection Report.pdf"  # Assign a default name if extension is invalid
                
                # Construct the full path for the file inside the "Images" folder
                full_file_name = os.path.join(Condition_report_folder, f"{reg_no}.pdf")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {Inspection_report_urls} for {reg_no}: {e}")

        # Assured_report_folder
        # Check if the Inspection_report_urls is missing
        # if not Assured_report_urls or pd.isna(Assured_report_urls):
        #     print(f"Missing Other Report of {reg_no}")
        #     # continue
        # else:
        
        #     if not Assured_report_urls.startswith(("http://", "https://")):
        #         Assured_report_urls = urljoin("https://", Assured_report_urls)  # Add 'https://' if missing

        #     # Parse the URL
        #     parsed_url = urlparse(Assured_report_urls)

        #     # Check if the parsed URL is valid
        #     if not parsed_url.scheme or not parsed_url.netloc:
        #         print(f"Invalid URL skipped: {Assured_report_urls}")
        #         continue
            
        #     # Try downloading the image
        #     try:
        #         # Download the image
        #         response = requests.get(Assured_report_urls, stream=True)
        #         response.raise_for_status()  # Raises an exception if the HTTP request fails

        #         # Extract the file name from the URL
        #         file_name = os.path.basename(parsed_url.path) or f"Other Report.pdf"

        #         # Check for valid file extensions
        #         file_extension = file_name.split(".")[-1]
        #         if file_extension not in ["pdf", "jpeg", "png", "gif", "bmp", "webp"]:
        #             file_name = f"Other Report.pdf"  # Assign a default name if extension is invalid
                
        #         # Construct the full path for the file inside the "Damaged" folder
        #         full_file_name = os.path.join(Assured_report_folder, f"{reg_no}.pdf")
                
        #         # Save the image
        #         with open(full_file_name, 'wb') as f:
        #             for chunk in response.iter_content(1024):
        #                 f.write(chunk)
                
        #         print(f"Downloaded: {full_file_name}")
        #     except Exception as e:
        #         print(f"Failed to download {Assured_report_urls} for {reg_no}: {e}")

def start_funcs():
    thread1 = threading.Thread(target=download_images, args=(reg_img,))
    thread2 = threading.Thread(target=download_reports, args=(reports,))
    
    # start threads 
    thread1.start()
    thread2.start()

    # join threads
    thread1.join()
    thread2.join()
    
if __name__ == "__main__":
    start_funcs() 

Watermark added to Images\PL07STU\Original_images\PL07STU_1.jpg
Downloaded: Images\PL07STU\Original_images\PL07STU_1.jpg
Downloaded: Reports\Inspection Report\PL07STU.pdf
Watermark added to Images\PL07STU\Original_images\PL07STU_2.jpg
Downloaded: Images\PL07STU\Original_images\PL07STU_2.jpg
Downloaded: Reports\Inspection Report\LD16RFO.pdf
Watermark added to Images\PL07STU\Original_images\PL07STU_3.jpg
Downloaded: Images\PL07STU\Original_images\PL07STU_3.jpg
Downloaded: Reports\Inspection Report\OV14YKB.pdf
Watermark added to Images\PL07STU\Original_images\PL07STU_4.jpg
Downloaded: Images\PL07STU\Original_images\PL07STU_4.jpg
Watermark added to Images\PL07STU\Original_images\PL07STU_5.jpg
Downloaded: Images\PL07STU\Original_images\PL07STU_5.jpg
Downloaded: Reports\Inspection Report\PN13CWA.pdf
Watermark added to Images\PL07STU\Original_images\PL07STU_6.jpg
Downloaded: Images\PL07STU\Original_images\PL07STU_6.jpg
Downloaded: Reports\Inspection Report\LC60XCG.pdf
Watermark added to Image